In [1]:
from tqdm import tqdm
import json
import pandas as pd

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
pages_folder = 'emetophobia_pages'
dld_folder = 'emetophobia_posts'

In [3]:
with open(f'{pages_folder}/emetophobia_all_pages.json', 'r') as f:
    all_pages = json.load(f)

len(all_pages)

996

title, content, date, upvote, downvote, labels

In [4]:
all_posts = []

for page in tqdm(all_pages):
    data = page['data']
    post = {
        'title': data['title'],
        'content': data['selftext'],
        'date': data['created_utc'],
        'ups': data['ups'],
        'upvote_ratio': data['upvote_ratio'],
        'downvotes': int(data['ups'] * (1 - data['upvote_ratio']) / data['upvote_ratio']),
        'labels': data['link_flair_richtext'],
    }

    all_posts.append(post)

with open(f'{dld_folder}/emetophobia_all_posts.json', 'w') as f:
    json.dump(all_posts, f, indent=4)

len(all_posts)

100%|██████████| 996/996 [00:00<00:00, 569610.96it/s]


996

In [5]:
all_todf = all_posts.copy()

for post in all_todf:
    label = 'None'

    for lbl in post['labels']:
        if lbl['e'] == 'text' and lbl['t'].strip() != '':
            label = lbl['t'].strip()
            break

    post['labels'] = label

all_todf[0]['labels'], len(all_todf)

('Techniques, tips and tricks', 996)

In [6]:
df_posts = pd.DataFrame.from_records(all_todf)
df_posts['date'] = pd.to_datetime(df_posts['date'], unit='s')

In [7]:
with open(f'{dld_folder}/emetophobia_all_posts_one_label.json', 'w') as f:
    json.dump(all_todf, f, indent=4)
df_posts.to_excel(f'{dld_folder}/emetophobia_all_posts.xlsx', index=False)

In [10]:
df_posts['labels'].value_counts()

labels
Question                                                      199
Needing support - Panic attack                                151
Rant                                                          135
Potentially Triggering                                         85
Does Anyone Else...?                                           78
Needing support: Just not feeling good                         73
Needing Support - In Acute Crisis (at risk of self injury)     45
Venting - Advice wanted                                        42
Needing Support - N, V, D etc                                  40
Success!                                                       39
Needing Support - Anxious about FP                             31
Techniques, tips and tricks                                    19
None                                                           16
It Happened (TW)                                               12
Recovery                                                       11
Pos

In [13]:
emoji_post = None
for post in all_todf:
    if post['title'] == 'Does anybody else??':
        print(post['content'])
        emoji_post = post['content']
        break



So I’m making this because I want to know if anyone agrees or relates but…. Do any of you guys refuse to use this emoji 🤮or 🤢because you’re scared that it will happen if you use these emojis??? Also when talking for example: say I wanted to go home and was telling my friend and they say “just fake being sick. Pretend you threw up” like I’m not sure why but I feel like if I did this I would jinx something….. idk anybody else relate? 


In [14]:
import emoji
emoji.demojize(emoji_post, language='en')

'\n\nSo I’m making this because I want to know if anyone agrees or relates but…. Do any of you guys refuse to use this emoji :face_vomiting:or :nauseated_face:because you’re scared that it will happen if you use these emojis??? Also when talking for example: say I wanted to go home and was telling my friend and they say “just fake being sick. Pretend you threw up” like I’m not sure why but I feel like if I did this I would jinx something….. idk anybody else relate? '

In [17]:
set(df_posts['labels'].unique())

{'Does Anyone Else...?',
 'Interesting info/Articles',
 'It Happened (TW)',
 'Moderator',
 'Needing Support - Anxious about FP',
 'Needing Support - In Acute Crisis (at risk of self injury)',
 'Needing Support - N, V, D etc',
 'Needing Support - N, V, D etc NO REASSURANCE',
 'Needing Support - Non-Emet related',
 'Needing support - Panic attack',
 'Needing support: Just not feeling good',
 'None',
 'Positive Reminder',
 'Potentially Triggering',
 'Question',
 'Rant',
 'Recovery',
 'Success!',
 'Techniques, tips and tricks',
 'Therapy info!',
 'Venting - Advice wanted',
 'Venting - No advice please'}

In [20]:
count_emojis = 0

labels = {k:0 for k in df_posts['labels'].unique()}

for post in all_todf:
    demojized = emoji.demojize(post['content'], language='en')
    if demojized != post['content']:
        count_emojis += 1
    labels[post['labels']] += 1

count_emojis, labels

(148,
 {'Techniques, tips and tricks': 19,
  'Question': 199,
  'Needing Support - In Acute Crisis (at risk of self injury)': 45,
  'Needing support: Just not feeling good': 73,
  'Needing support - Panic attack': 151,
  'Venting - Advice wanted': 42,
  'Does Anyone Else...?': 78,
  'Rant': 135,
  'None': 16,
  'Potentially Triggering': 85,
  'Needing Support - N, V, D etc': 40,
  'Therapy info!': 3,
  'It Happened (TW)': 12,
  'Success!': 39,
  'Interesting info/Articles': 3,
  'Needing Support - Anxious about FP': 31,
  'Positive Reminder': 5,
  'Recovery': 11,
  'Needing Support - Non-Emet related': 4,
  'Venting - No advice please': 1,
  'Needing Support - N, V, D etc NO REASSURANCE': 3,
  'Moderator': 1})